In [ ]:
!pip install spacy -U
!python -m spacy download pl_core_news_sm

     |████████████████████████████████| 58.6 MB 1.3 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('pl_core_news_sm')


In [ ]:
!pip install rich

In [ ]:
import re
import regex as re
import spacy
import pandas as pd
import numpy as np
import spacy
import os
from tqdm import tqdm
from nltk.tokenize import word_tokenize
import nltk


In [ ]:
from rich import inspect
import pandas as pd

In [ ]:
inspect(pd.DataFrame, methods = True)

╭────────────────────────── <class 'pandas.core.frame.DataFrame'> ──────────────────────────╮
│ class DataFrame(data=None, index: 'Axes | None' = None, columns: 'Axes | None' = None,    │
│ dtype: 'Dtype | None' = None, copy: 'bool | None' = None):                                │
│                                                                                           │
│ Two-dimensional, size-mutable, potentially heterogeneous tabular data.                    │
│                                                                                           │
│                at = <property object at 0x7f93f8b763b0>                                   │
│             attrs = <property object at 0x7f93f89e93b0>                                   │
│              axes = <property object at 0x7f93f8882bf0>                                   │
│           columns = <pandas._libs.properties.AxisProperty object at 0x7f93f881ed10>       │
│            dtypes = <property object at 0x7f93f8932d10>                                   │
│             empty = <property object at 0x7f93f8932b30>                                   │
│             flags = <property object at 0x7f93f8bd6cb0>                                   │
│               iat = <property object at 0x7f93f8b76410>                                   │
│              iloc = <property object at 0x7f93f8b762f0>                                   │
│             index = <pandas._libs.properties.AxisProperty object at 0x7f93f8808890>       │
│               loc = <property object at 0x7f93f8b76350>                                   │
│              ndim = <property object at 0x7f93f8932a70>                                   │
│             shape = <property object at 0x7f93f8873530>                                   │
│              size = <property object at 0x7f93f8932ad0>                                   │
│             style = <property object at 0x7f93f880cbf0>                                   │
│                 T = <property object at 0x7f93f880cc50>                                   │
│            values = <property object at 0x7f93f88201d0>                                   │
│               abs = def abs(self: 'FrameOrSeries') -> 'FrameOrSeries': Return a           │
│                     Series/DataFrame with absolute numeric value of each element.         │
│               add = def add(self, other, axis='columns', level=None, fill_value=None):    │
│                     Get Addition of dataframe and other, element-wise (binary operator    │
│                     `add`).                                                               │
│        add_prefix = def add_prefix(self: 'FrameOrSeries', prefix: 'str') ->               │
│                     'FrameOrSeries': Prefix labels with string `prefix`.                  │
│        add_suffix = def add_suffix(self: 'FrameOrSeries', suffix: 'str') ->               │
│                     'FrameOrSeries': Suffix labels with string `suffix`.                  │
│               agg = def agg(self, func=None, axis: 'Axis' = 0, *args, **kwargs):          │
│                     Aggregate using one or more operations over the specified axis.       │
│         aggregate = def aggregate(self, func=None, axis: 'Axis' = 0, *args, **kwargs):    │
│                     Aggregate using one or more operations over the specified axis.       │
│             align = def align(self, other, join: 'str' = 'outer', axis: 'Axis | None' =   │
│                     None, level: 'Level | None' = None, copy: 'bool' = True,              │
│                     fill_value=None, method: 'str | None' = None, limit=None, fill_axis:  │
│                     'Axis' = 0, broadcast_axis: 'Axis | None' = None) -> 'DataFrame':     │
│                     Align two objects on their axes with the specified join method.       │
│               all = def all(self, axis=0, bool_only=None, skipna=True, level=None,        │
│                     **kwargs): Return whether all 

Część 1- przetwarzanie i czyszczenie danych

In [ ]:
my_dataframe = pd.read_csv("/content/all_tweets_id.csv")
opis = my_dataframe.describe(); #chce pokazać opis obiektu
opis 
for col in my_dataframe.columns: #pokazuje wszystkie kolumny
    print(col)

coordinates
created_at
hashtags
media
urls
favorite_count
id
in_reply_to_screen_name
in_reply_to_status_id
in_reply_to_user_id
lang
place
possibly_sensitive
quote_id
retweet_count
retweet_id
retweet_screen_name
source
text
tweet_url
user_created_at
user_id
user_default_profile_image
user_description
user_favourites_count
user_followers_count
user_friends_count
user_listed_count
user_location
user_name
user_screen_name
user_statuses_count
user_time_zone
user_urls
user_verified


In [ ]:
def dni_tygodnia_i_miesiace(wartosc): #za jednym zamachem chce zrobić dni tygodnia na PL a  miesiące na liczby
    tmp = str(wartosc).replace("Mon", "Pon").replace("Tue", "Wt").replace("Wed", "Sr").replace("Thu", "Czw").replace("Fri", "Pt").replace("Sat", "Sob").replace("Sun", "Niedz").replace("Jan", "01").replace("Feb", "02").replace("Mar", "03").replace("Apr", "04").replace("May", "05").replace("June", "06").replace("July", "07").replace("Aug", "08").replace("Sept", "09").replace("Oct", "10") .replace("Nov", "11").replace("Dec", "12")
    return tmp

my_dataframe["created_at"] = my_dataframe["created_at"].apply(dni_tygodnia_i_miesiace)

print(my_dataframe["created_at"])


0        Wt 11 02 18:47:39 +0000 2021
1        Wt 11 02 18:29:49 +0000 2021
2        Wt 11 02 18:35:58 +0000 2021
3        Wt 11 02 18:50:36 +0000 2021
4        Wt 11 02 18:31:32 +0000 2021
                    ...              
6315    Pon 10 25 09:25:53 +0000 2021
6316    Pon 10 25 04:14:32 +0000 2021
6317    Pon 10 25 09:03:09 +0000 2021
6318    Pon 10 25 06:45:14 +0000 2021
6319    Pon 10 25 00:29:16 +0000 2021
Name: created_at, Length: 6320, dtype: object


In [ ]:
url_lista = list(my_dataframe["tweet_url"]) #pobieram linki do tweetów
print(url_lista)

['https://twitter.com/ArturEichhorst/status/1455607555816833031', 'https://twitter.com/biedronkissa/status/1455603066250612740', 'https://twitter.com/Jarko_22/status/1455604614162751493', 'https://twitter.com/waldema61437880/status/1455608295155281921', 'https://twitter.com/DOrpych/status/1455603496955301895', 'https://twitter.com/KrzyzanekMarek/status/1455602402678218760', 'https://twitter.com/vocatus_/status/1455601213903409159', 'https://twitter.com/pigmalion55/status/1455607715989016587', 'https://twitter.com/mario_wsh/status/1455604289418711045', 'https://twitter.com/yrizona_/status/1455602391982743554', 'https://twitter.com/GrEnaroc/status/1455605810730577923', 'https://twitter.com/MarcCoufal/status/1455602283425697797', 'https://twitter.com/4rmar/status/1455604783499333642', 'https://twitter.com/Ewa_Jedrych/status/1455603797817020421', 'https://twitter.com/LewaStronaMocy/status/1455600266422669316', 'https://twitter.com/BEATAMartynows1/status/1455601057556570118', 'https://twitt

In [ ]:
tmp_df = my_dataframe.dropna(subset=['urls']) #pojawiły się nan- trzeba je usunąć za pomocą funkcji dropna
linki_lista = list(tmp_df["urls"])
print(linki_lista)

['https://twitter.com/tom_wiejski/status/1455582512953151498', 'https://oko.press/ku-pamieci-kobiety-ktora-umarla-tylko-dlatego-ze-byla-w-ciazy/ https://oko.press/lista-hanby-oto-poslowie-pis-i-konfederacji-ktorzy-doprowadzili-do-zakazu-aborcji-w-polsce/', 'https://oko.press/ku-pamieci-kobiety-ktora-umarla-tylko-dlatego-ze-byla-w-ciazy/ https://oko.press/lista-hanby-oto-poslowie-pis-i-konfederacji-ktorzy-doprowadzili-do-zakazu-aborcji-w-polsce/', 'https://oko.press/ku-pamieci-kobiety-ktora-umarla-tylko-dlatego-ze-byla-w-ciazy/ https://oko.press/lista-hanby-oto-poslowie-pis-i-konfederacji-ktorzy-doprowadzili-do-zakazu-aborcji-w-polsce/', 'https://www.politykazdrowotna.com/71080,aborcja-stala-sie-praktycznie-niedopuszczalna-w-polsce', 'https://twitter.com/somsiadnosacz/status/1455501997688475649', 'https://wiadomosci.radiozet.pl/Polska/Aborcja-zrownana-z-morderstwem.-Sejm-zajmie-sie-nowym-projektem-zakazujacym-przerywania-ciazy', 'https://kobieta.wp.pl/12-lat-temu-trafila-do-szpitala-z-b

In [ ]:
tmp_sort_df = my_dataframe.sort_values("media") #eliminujemy duplikaty, musimy jednak wcześniej posortować dane
tmp_bez_dup = tmp_sort_df.drop_duplicates(subset ="media")

media_lista = list(tmp_bez_dup["media"])
print(linki_lista)

['https://twitter.com/tom_wiejski/status/1455582512953151498', 'https://oko.press/ku-pamieci-kobiety-ktora-umarla-tylko-dlatego-ze-byla-w-ciazy/ https://oko.press/lista-hanby-oto-poslowie-pis-i-konfederacji-ktorzy-doprowadzili-do-zakazu-aborcji-w-polsce/', 'https://oko.press/ku-pamieci-kobiety-ktora-umarla-tylko-dlatego-ze-byla-w-ciazy/ https://oko.press/lista-hanby-oto-poslowie-pis-i-konfederacji-ktorzy-doprowadzili-do-zakazu-aborcji-w-polsce/', 'https://oko.press/ku-pamieci-kobiety-ktora-umarla-tylko-dlatego-ze-byla-w-ciazy/ https://oko.press/lista-hanby-oto-poslowie-pis-i-konfederacji-ktorzy-doprowadzili-do-zakazu-aborcji-w-polsce/', 'https://www.politykazdrowotna.com/71080,aborcja-stala-sie-praktycznie-niedopuszczalna-w-polsce', 'https://twitter.com/somsiadnosacz/status/1455501997688475649', 'https://wiadomosci.radiozet.pl/Polska/Aborcja-zrownana-z-morderstwem.-Sejm-zajmie-sie-nowym-projektem-zakazujacym-przerywania-ciazy', 'https://kobieta.wp.pl/12-lat-temu-trafila-do-szpitala-z-b

In [ ]:
url_obrazki_lista = list(my_dataframe["media"]) #linki do obrazków
print(url_obrazki_lista)

[nan, 'https://twitter.com/NocnaZ/status/1455570500785168396/photo/1', nan, nan, 'https://twitter.com/NocnaZ/status/1455570500785168396/photo/1', nan, nan, 'https://twitter.com/NocnaZ/status/1455570500785168396/photo/1', nan, nan, nan, nan, nan, nan, nan, nan, 'https://twitter.com/NocnaZ/status/1455570500785168396/photo/1', nan, nan, 'https://twitter.com/NocnaZ/status/1455570500785168396/photo/1', nan, nan, nan, nan, nan, nan, nan, 'https://twitter.com/NocnaZ/status/1455570500785168396/photo/1', 'https://twitter.com/boklazec/status/1455602774704599041/video/1', nan, nan, 'https://twitter.com/NocnaZ/status/1455570500785168396/photo/1', 'https://twitter.com/NocnaZ/status/1455570500785168396/photo/1', 'https://twitter.com/NocnaZ/status/1455570500785168396/photo/1', nan, nan, 'https://twitter.com/NocnaZ/status/1455570500785168396/photo/1', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 'https://twitter.com/NocnaZ/status/1455570500785168396/photo/1', nan, nan, na

In [ ]:
tmp_df = my_dataframe.dropna(subset=['media']) #podobnie jak wcześniej pojawiły się nan
obrazki_lista = list(tmp_df['media'])
print(obrazki_lista)

['https://twitter.com/NocnaZ/status/1455570500785168396/photo/1', 'https://twitter.com/NocnaZ/status/1455570500785168396/photo/1', 'https://twitter.com/NocnaZ/status/1455570500785168396/photo/1', 'https://twitter.com/NocnaZ/status/1455570500785168396/photo/1', 'https://twitter.com/NocnaZ/status/1455570500785168396/photo/1', 'https://twitter.com/NocnaZ/status/1455570500785168396/photo/1', 'https://twitter.com/boklazec/status/1455602774704599041/video/1', 'https://twitter.com/NocnaZ/status/1455570500785168396/photo/1', 'https://twitter.com/NocnaZ/status/1455570500785168396/photo/1', 'https://twitter.com/NocnaZ/status/1455570500785168396/photo/1', 'https://twitter.com/NocnaZ/status/1455570500785168396/photo/1', 'https://twitter.com/NocnaZ/status/1455570500785168396/photo/1', 'https://twitter.com/NocnaZ/status/1455570500785168396/photo/1', 'https://twitter.com/NocnaZ/status/1455570500785168396/photo/1', 'https://twitter.com/NocnaZ/status/1455570500785168396/photo/1', 'https://twitter.com/N

In [ ]:
pl_model = spacy.load('pl_core_news_sm') #usuwamy stopwords za pomocą polskiego spacy
nltk.download('punkt')
stopwords = pl_model.Defaults.stop_words
text_without_stopwords = []

for wiersz in my_dataframe['text']:
  wiersz_tokens = word_tokenize(wiersz)
  wiersz_bez_sw = [slowo for slowo in wiersz_tokens if not slowo in stopwords]
  text_without_stopwords.append(wiersz_bez_sw)

my_dataframe['text_without_stopwords'] = text_without_stopwords
print(my_dataframe['text_without_stopwords'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
0       [Dziś, @, radekfogiel, powiedział, ,, prac, ab...
1       [#, AniJednejWięcej, Aborcja, granic, :, +48, ...
2       [Sejm, planuje, obrady, projektem, zakazującym...
3       [Sejm, planuje, obrady, projektem, zakazującym...
4       [#, AniJednejWięcej, Aborcja, granic, :, +48, ...
                              ...                        
6315    [Jak, wojna, Czechami, ., Korzyści, przegranej...
6316    [@, JoankaSW, @, __Lewica, @, Lewica_News, @, ...
6317    [``, Wyjście, UE, utrudniłby, wyjazdy, granicę...
6318    [@, PolskaKobieta, @, MarekWivaxxa, Martusi, u...
6319    [@, Cat_Ewa2, @, mariuszem1, @, dabrowsk10, @,...
Name: text_without_stopwords, Length: 6320, dtype: object


Część 2- analiza danych

In [ ]:
tmp_sort_df = my_dataframe.sort_values("user_favourites_count", ascending=False ) #sortujemy top 5 polubień
print(tmp_sort_df.head(5)["text"])

5244    To już jawne skur*ysyństwo!\nDwie lewackie dzi...
5906    @DaleborL @AlfaBies Nie w tym rzecz, że aborcj...
1876    Moja ciąża jest moją sprawą. \nMoja aborcja je...
876     #PAMIĘTAMY\n\nPrzeczytaj więcej: https://t.co/...
5521    Gdyby ministranci zachodzili w ciążę, aborcja ...
Name: text, dtype: object


In [ ]:
tmp_sort_df = my_dataframe.sort_values("retweet_count", ascending=False ) #sortujemy top 5 retwettów
tmp_bez_dup = tmp_sort_df.drop_duplicates(subset ="text")

print(tmp_bez_dup.head(5)["text"])

1310    #anijednejwiecej\nOto lista 118 posłanek i pos...
4173    Po wczorajszym twierdzę, że dla rozjuszonej dz...
6184    Jak już ma być jakaś wojna to niech będzie z C...
336     Dziś to ważne jeszcze bardziej. \n\nGdziekolwi...
5125    To już jawne skur*ysyństwo!\nDwie lewackie dzi...
Name: text, dtype: object


In [ ]:
my_dataframe.loc[my_dataframe['possibly_sensitive'] == 'False']


,coordinates,created_at,hashtags,media,urls,favorite_count,id,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_user_id,lang,place,possibly_sensitive,quote_id,retweet_count,retweet_id,retweet_screen_name,source,text,tweet_url,user_created_at,user_id,user_default_profile_image,user_description,user_favourites_count,user_followers_count,user_friends_count,user_listed_count,user_location,user_name,user_screen_name,user_statuses_count,user_time_zone,user_urls,user_verified,text_without_stopwords


In [ ]:
from datetime import datetime

ile_uplynelo = []
#liczymy ile uplynelo od wybranej daty do zalozenia konta
for wiersz in my_dataframe['user_created_at']:
  ile_uplynelo.append((datetime.strptime(wiersz, "%a %b %d %H:%M:%S %z %Y")- datetime.strptime("Thu Jan 1 00:00:00 +0000 1970", "%a %b %d %H:%M:%S %z %Y")).days)

tmp_df = my_dataframe
tmp_df['ile_uplynelo'] = ile_uplynelo

user = tmp_df.sort_values('ile_uplynelo', ascending=True ).head(1)['user_id'].values[0]
print(user)

my_dataframe.loc[my_dataframe['user_id'] == user] #user ma tylko jednego tweeta

9035752


,coordinates,created_at,hashtags,media,urls,favorite_count,id,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_user_id,lang,place,possibly_sensitive,quote_id,retweet_count,retweet_id,retweet_screen_name,source,text,tweet_url,user_created_at,user_id,user_default_profile_image,user_description,user_favourites_count,user_followers_count,user_friends_count,user_listed_count,user_location,user_name,user_screen_name,user_statuses_count,user_time_zone,user_urls,user_verified,text_without_stopwords,ile_uplynelo
4290,NaN,Niedz 10 31 12:06:00 +0000 2021,NaN,NaN,NaN,0,1454781698139430917,NaN,NaN,NaN,pl,NaN,NaN,NaN,78,1.454779e+18,kamila_ferenc,"<a href=""http://twitter.com/download/android"" ...","Jak powiesz 'aborcja to morderstwo', w k.k. wp...",https://twitter.com/Hazardius/status/145478169...,Sat Sep 22 18:01:31 +0000 2007,9035752,False,31🇬🇱 @ 🇳🇴Bergen - 🏳️‍🌈🍳ASD\nC♯🎶/Ruby♦️/Python🐍...,98564,379,1340,3,"Bergen, Norge",Hazardius 🡗🡗🡗,Hazardius,28429,NaN,NaN,False,"[Jak, powiesz, 'aborcja, morderstwo, ', ,, k.k...",13778


In [ ]:
user = my_dataframe.sort_values('user_followers_count', ascending=False ).head(1)['user_id'].values[0]
print(user)

my_dataframe.loc[my_dataframe['user_id'] == user] #najwięcej followersów i tylko jeden tweet

14493047


,coordinates,created_at,hashtags,media,urls,favorite_count,id,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_user_id,lang,place,possibly_sensitive,quote_id,retweet_count,retweet_id,retweet_screen_name,source,text,tweet_url,user_created_at,user_id,user_default_profile_image,user_description,user_favourites_count,user_followers_count,user_friends_count,user_listed_count,user_location,user_name,user_screen_name,user_statuses_count,user_time_zone,user_urls,user_verified,text_without_stopwords,ile_uplynelo
1352,NaN,Wt 11 02 13:00:27 +0000 2021,NaN,NaN,https://tvn24.pl/polska/aborcja-sejm-zajmie-si...,28,1455520178943451141,NaN,NaN,NaN,pl,NaN,False,NaN,18,NaN,NaN,"<a href=""https://about.twitter.com/products/tw...",Obywatelski projekt zaostrzający kary za aborc...,https://twitter.com/tvn24/status/1455520178943...,Wed Apr 23 13:12:25 +0000 2008,14493047,False,Najnowsze informacje z Polski i ze świata.,51,1548727,68,1921,Poland,tvn24,tvn24,324446,NaN,http://www.tvn24.pl,True,"[Obywatelski, projekt, zaostrzający, kary, abo...",13992


In [ ]:
my_dataframe.loc[my_dataframe['user_verified'] == 'True']

,coordinates,created_at,hashtags,media,urls,favorite_count,id,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_user_id,lang,place,possibly_sensitive,quote_id,retweet_count,retweet_id,retweet_screen_name,source,text,tweet_url,user_created_at,user_id,user_default_profile_image,user_description,user_favourites_count,user_followers_count,user_friends_count,user_listed_count,user_location,user_name,user_screen_name,user_statuses_count,user_time_zone,user_urls,user_verified,text_without_stopwords,ile_uplynelo


In [ ]:
dni_pub = []

for wiersz in my_dataframe['created_at']:
  dni_pub.append(wiersz[:3])

tmp_df = my_dataframe
tmp_df['dni_pub'] = dni_pub
tmp_df['dni_pub'].describe() #Wtorek najpopularniejszy

count     6320
unique       7
top        Wt 
freq      3078
Name: dni_pub, dtype: object

Część 3 

In [ ]:
my_dataframe['text']  

0       Dziś @radekfogiel powiedział, że nie ma żadnyc...
1       #AniJednejWięcej \nAborcja bez granic: +48 22 ...
2       Sejm planuje obrady nad projektem zakazującym ...
3       Sejm planuje obrady nad projektem zakazującym ...
4       #AniJednejWięcej \nAborcja bez granic: +48 22 ...
                              ...                        
6315    Jak już ma być jakaś wojna to niech będzie z C...
6316    @JoankaSW @__Lewica @Lewica_News @mlodalewica_...
6317    "Wyjście z UE utrudniłby wyjazdy za granicę, k...
6318    @PolskaKobieta @MarekWivaxxa Martusi nie udała...
6319    @Cat_Ewa2 @mariuszem1 @dabrowsk10 @najeznik635...
Name: text, Length: 6320, dtype: object

In [ ]:
import spacy

In [ ]:
from spacy import displacy
nlp = spacy.load("pl_core_news_sm")
tekst = my_dataframe['text']

my_dataframe['persons'] = np.empty((len(my_dataframe), 0)).tolist()
i = 0
for wiersz in my_dataframe['text']:
  doc = nlp(wiersz)
  # Zidentyfikuj persony
  tmp_persons = [ent.text for ent in doc.ents if ent.label_ == 'persName']
  # persons.append(persons)
  my_dataframe['persons'][i] = tmp_persons
  i = i+1
  if (i == 40):
    break #zostawiam dla pierwszych 40, bo wywala sie RAM



my_dataframe['persons'][5]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


['Julia']

In [37]:
from spacy import displacy
nlp = spacy.load("pl_core_news_sm")
tekst = my_dataframe['text']

my_dataframe['places'] = np.empty((len(my_dataframe), 0)).tolist()
i = 0
for wiersz in my_dataframe['text']:
  doc = nlp(wiersz)
  # Zidentyfikuj miejsaca
  tmp_places = [ent.text for ent in doc.ents if ent.label_ == 'placeName']
  my_dataframe['places'][i] = tmp_places
  i = i+1
  if (i == 40):
    break #zostawiam dla pierwszych 40, bo wywala sie RAM



my_dataframe['places']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


0                         [@radekfogiel, UstawaRatunkowa]
1                                                      []
2       [Polska, Salwador, Nikaragua, Honduras, Domini...
3       [Polska, Salwador, Nikaragua, Honduras, Domini...
4                                                      []
                              ...                        
6315                                                   []
6316                                                   []
6317                                                   []
6318                                                   []
6319                                                   []
Name: places, Length: 6320, dtype: object

In [38]:
from spacy import displacy
nlp = spacy.load("pl_core_news_sm")
tekst = my_dataframe['text']

my_dataframe['organizations'] = np.empty((len(my_dataframe), 0)).tolist()
i = 0
for wiersz in my_dataframe['text']:
  doc = nlp(wiersz)
  # Zidentyfikuj organizacje
  tmp_organizations = [ent.text for ent in doc.ents if ent.label_ == 'orgName']
  my_dataframe['organizations'][i] = tmp_organizations
  i = i+1
  if (i == 40):
    break #zostawiam dla pierwszych 40, bo wywala sie RAM



my_dataframe['organizations']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


0                    []
1                    []
2       [Sejm, Watykan]
3       [Sejm, Watykan]
4                    []
             ...       
6315                 []
6316                 []
6317                 []
6318                 []
6319                 []
Name: organizations, Length: 6320, dtype: object